# 用Bokeh观察多元函数

In [42]:
from sympy import var, lambdify, sin

In [43]:
variables = var("x y z")
f =  sin(x*y + 2*y*z + 3*z*x)

In [44]:
from sympyhelp import to_javascript
from IPython.display import display_javascript

In [51]:
display_javascript(to_javascript("func_f", variables, f), raw=True)
func_f = lambdify(variables, f, modules="numpy")

In [52]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Slider, HoverTool
from bokeh.models.callbacks import CustomJS
from bokeh.layouts import widgetbox, column, row
from bokeh.core import properties
from bokeh.io import output_notebook, show
import numpy as np
output_notebook()

Loading BokehJS ...

In [56]:
ranges = {
    x: (-2, 2, 0.1),
    y: (-2, 2, 0.1),
    z: (-2, 2, 0.1),
}
npoints = 300
figs = [figure(plot_width=300, plot_height=150, logo=None, tools=["pan", "box_zoom", "wheel_zoom"]) for v in variables]
sliders = [Slider(name=str(v), title=str(v), format="0[.]00000",
                  start=ranges[v][0], end=ranges[v][1], value=ranges[v][2],
                  step = (ranges[v][1] - ranges[v][0]) / 20
                 ) for v in variables]
wbox = widgetbox(sliders)

data = {}
for v in variables:
    xdata = np.linspace(ranges[v][0], ranges[v][1], npoints)
    args = [xdata if v is v2 else ranges[v2][2] for v2 in variables]
    data[str(v)] = xdata
    ydata = func_f(*args)
    data["f_{}".format(v)] = ydata

source = ColumnDataSource(data=data)

for v, fig in zip(variables, figs):
    fig.line(str(v), "f_{}".format(v), source=source, line_color="blue")
    fig.xaxis.axis_label = str(v)
    
def callback_func(source=source, sliders=wbox):
    data = source.data
    for i, slider in enumerate(sliders.children):
        args = [slider.value for slider in sliders.children]
        for j in range(len(data[slider.name])):
            args[i] = data[slider.name][j]
            f_name = "f_" + slider.name
            data[f_name][j] = window["func_f"](args)
    source.change.emit()

callback = CustomJS.from_py_func(callback_func)

for slider in sliders:
    slider.js_on_change("value", callback)

show(row(column(figs), wbox))